In [18]:
import torch
import random
import numpy as np

In [ ]:
x1 = torch.tensor([1.0]).double(); x1.requires_grad = True
w1 = torch.tensor([5.0]).double(); w1.requires_grad = True
x2 = torch.tensor([2.0]).double(); x2.requires_grad = True
w2 = torch.tensor([3.0]).double(); w2.requires_grad = True
b = torch.tensor([4.0]).double(); b.requires_grad = True
o = x1 * w1 + x2 * w2 + b
print(o.data)
o.backward()

print(x1.grad.data)
print(w1.data)
print(x2.data)
print(w2.data)


tensor([15.], dtype=torch.float64)
tensor([5.], dtype=torch.float64)
tensor([5.], dtype=torch.float64)
tensor([2.], dtype=torch.float64)
tensor([3.], dtype=torch.float64)


In [54]:
# Creating a neuron
class Neuron:

  def __init__(self, n_in):
    self.w = np.array([random.uniform(-1, 1) for _ in range (n_in)])
    self.b = random.uniform(-1, 1)

  def __call__(self, x):
    activation = np.dot(self.w, x) + self.b
    output = np.tanh(activation)
    return output.item()

# Creating a layer
class Layer:

  def __init__(self, n_in, n_out):
    self.neurons = [Neuron(n_in) for _ in range(n_out)]

  def __call__(self, x):
    outputs = [n(x) for n in self.neurons]
    return outputs

# Creating a multi-layer perceptron
class MLP:

    def __init__(self, n_in, n_outs):
      size = [n_in] + n_outs
      self.layers = [Layer(size[i], size[i+1]) for i in range(len(n_outs))]

    def __call__(self, x):
      for layer in self.layers:
        x = layer(x)
      return x

x = np.array([2.0, 3.0, 5.0])
n = MLP(3, [4, 4, 1])
n(x)

[0.185614445985857]